In [13]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [14]:
# Stocks on the s&p500 list, stored in a csv file on the pc
stocks = pd.read_csv('sp_500_stocks.csv') 

In [15]:
# IEX Cloud API token to access random data freely and try the algorithms
from secrets import IEX_CLOUD_API_TOKEN

In [21]:
""""
symbol: Sample stock ticker to try out stuff
api_url: URL to access data on said stock
data: The request to get data on said stock
"""

symbol='AAPL' 
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()


In [23]:
# Simple short test
price = data['latestPrice']
market_cap = data['marketCap']

2422297102285


In [24]:
"""
my_columns: List of keys for the F_DataFrame
F_DataFrame: pandas DataFrame formed by 4 columns containing all the relevant data on the stock
"""

my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
F_DataFrame = pd.DataFrame([[0,0,0,0]], columns= my_columns)

"""
Testing wether this is taking the shape we are interested in
"""
F_DataFrame.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index = True #Common error when forming a pandas.DataFrame
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,0,0.00,0,0
1,AAPL,142.38,2422297102285,N/A


In [29]:
""" 
DataFrame to handle all the stocks
"""
F_DataFrame = pd.DataFrame(columns= my_columns)


"""
for loop to get all the data
"""
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    F_DataFrame = F_DataFrame.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns),
    ignore_index=True
    )
    

In [30]:
"""
Getting all the info in 1 request is really slow, so better get it in chunks
Starting by chopping the list of tickers
"""

def chunks(lst, n):
    """chunks of size en off the list"""
    for i in range(0, len(lst), n):
        yield lst[i: i+n]

In [45]:
symbol_groups = list(chunks(stocks['Ticker'], 100))

symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
final_dataframe = pd.DataFrame(columns = my_columns)


""" For the batch call we make a very similar call to the previous one """

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
        ],
        index = my_columns),
        ignore_index=True
        )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,159.42,48187117633,N/A
1,AAL,20.56,13915919301,N/A
2,AAP,216.50,13574565761,N/A
3,AAPL,143.00,2434924474584,N/A
4,ABBV,109.89,196484570777,N/A
...,...,...,...,...
500,YUM,124.82,36685607621,N/A
501,ZBH,151.88,31450363289,N/A
502,ZBRA,525.41,28219800855,N/A
503,ZION,65.90,10252339023,N/A


In [46]:
"""Amount of money available"""

portfolio_size = input("Enter the value of your portfolio")


""" This try-except method should deal with the eventual phrase input
     by the user"""
try: 
    val = float(portfolio_size)
except ValueError:
    print("That's not a number you dumbo\n Try again:")
    portfolio_size = input("Enter the value of your portfolio: ")

Enter the value of your portfolio1000000


In [54]:
""" Amount of money put in each individual stock
    Fractions of share could come up, to be handled in the next operation"""

position_size = float(portfolio_size) / len(final_dataframe.index)


""" Now allocate in the 'Number of Shares to Buy' column the number of total
    shares, rounded down so as to not go over board with the potfolio_size """

for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,159.42,48187117633,12
1,AAL,20.56,13915919301,96
2,AAP,216.50,13574565761,9
3,AAPL,143.00,2434924474584,13
4,ABBV,109.89,196484570777,18
...,...,...,...,...
500,YUM,124.82,36685607621,15
501,ZBH,151.88,31450363289,13
502,ZBRA,525.41,28219800855,3
503,ZION,65.90,10252339023,30


In [65]:
writer = pd.ExcelWriter('recommended-trades.xlsx', engine = 'xlsxwriter')

final_dataframe.to_excel(writer, 'Recommended trades', index = False)


In [66]:
background_colour = '#0a0a23'
font_colour = '#ffffff'


string_format = writer.book.add_format(
    {
        'font_color' : font_colour,
        'bg_color' : background_colour,
        'border' : 1
    }
)


dollar_format = writer.book.add_format(
    {
        'num_format' : '$0.00',
        'font_color' : font_colour,
        'bg_color' : background_colour,
        'border' : 1
    }
)


integer_format = writer.book.add_format(
    {
        'num_format' : '0',
        'font_color' : font_colour,
        'bg_color' : background_colour,
        'border' : 1
    }
)



In [67]:
""" 
First build a dictionary specifying the format of each column
"""
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}


"""Now to build a loop that applies the formats to the intended columns"""

for column in column_formats.keys():
    writer.sheets['Recommended trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended trades'].write(f'{column}1', column_formats[column][0], string_format)
    
writer.save()